# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [ ]:
!pip install ratsnlp -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.8 MB/s eta 0:00:00


# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [ ]:
from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner2",
    max_seq_length=64,
)

# /gdrive/MyDrive/nlpbook/checkpoint-ner 안에
# lightning_logs, epoch=0-val_loss=0.12.ckpt 이건 가중치.., label_map.txt 이 세가지가 있어야함.

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/epoch=0-val_loss=0.12.ckpt
downstream_model_labelmap_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/label_map.txt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [ ]:
import torch
from transformers import BertConfig, BertForTokenClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
# 마지막 레이어의 바이어스(그리고 가중치)의 크기는 모델이 분류해야 하는 클래스의 수를 직접적으로 반영
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForTokenClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()
# ckpt는 훈련된 모델의 상태를 저장하거나 불러오는 데 사용되는 체크포인트 파일을 의미
# eval은 모델을 평가 모드로 설정하여 일관된 성능을 내도록 하는 데 사용

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 레이블 맵 작성

범주 인덱스를 범주명과 매칭하는 사전을 만듭니다.

- 문자: (선택 사항) 주어진 문자는 원래 문자열의 시작 또는 끝에서 제거됩니다.
- 문자 매개변수가 제공되지 않으면 문자열의 시작과 끝에서 공백이 제거됩니다.

In [ ]:
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [ ]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        probs = outputs.logits[0].softmax(dim=1)
        top_probs, preds = torch.topk(probs, dim=1, k=1) # 예측 값에서 top-k개의 결과를 받고 싶을 때
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        predicted_tags = [id_to_label[pred.item()] for pred in preds]
        result = []
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result = {
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)
    return {
        "sentence": sentence,
        "result": result,
    }

In [ ]:
# sentence = '코리아IT아카데미 학생들은 프로젝트 기반 AI 빅데이터 솔루션을 학습한다.'
sentence = "한국의 윤석렬 대통령은 어제 오후 5시에 서울에 있는 it회사를 방문해서 경제 상황을 점검했다. "
inference_fn(sentence)

{'sentence': '한국의 윤석렬 대통령은 어제 오후 5시에 서울에 있는 it회사를 방문해서 경제 상황을 점검했다. ',
 'result': [{'token': '한국의', 'predicted_tag': '기관명', 'top_prob': '0.908'},
  {'token': '윤석렬', 'predicted_tag': '인명', 'top_prob': '0.9774'},
  {'token': '대통령은', 'predicted_tag': 'O', 'top_prob': '0.9973'},
  {'token': '어제', 'predicted_tag': 'O', 'top_prob': '0.9992'},
  {'token': '오후', 'predicted_tag': '시간', 'top_prob': '0.9948'},
  {'token': '5', 'predicted_tag': '시간', 'top_prob': '0.9926'},
  {'token': '##시에', 'predicted_tag': '시간', 'top_prob': '0.9943'},
  {'token': '서울에', 'predicted_tag': '지명', 'top_prob': '0.9349'},
  {'token': '있는', 'predicted_tag': 'O', 'top_prob': '0.9979'},
  {'token': 'i', 'predicted_tag': 'O', 'top_prob': '0.5508'},
  {'token': '##t', 'predicted_tag': 'O', 'top_prob': '0.7303'},
  {'token': '##회사', 'predicted_tag': 'O', 'top_prob': '0.8902'},
  {'token': '##를', 'predicted_tag': 'O', 'top_prob': '0.9991'},
  {'token': '방문', 'predicted_tag': 'O', 'top_prob': '0.9998'},
  {'token': '##해서', 'p

In [ ]:
sentence = '기상청은 오늘 12시 날씨가 30도를 넘어갈 것이라고 예측한다'
inference_fn(sentence)

{'sentence': '기상청은 오늘 12시 날씨가 30도를 넘어갈 것이라고 예측한다',
 'result': [{'token': '기상청', 'predicted_tag': '기관명', 'top_prob': '0.9949'},
  {'token': '##은', 'predicted_tag': 'O', 'top_prob': '0.7295'},
  {'token': '오늘', 'predicted_tag': 'O', 'top_prob': '0.997'},
  {'token': '12', 'predicted_tag': '시간', 'top_prob': '0.9714'},
  {'token': '##시', 'predicted_tag': '시간', 'top_prob': '0.9805'},
  {'token': '날씨', 'predicted_tag': 'O', 'top_prob': '0.9943'},
  {'token': '##가', 'predicted_tag': 'O', 'top_prob': '0.9993'},
  {'token': '30', 'predicted_tag': '기타 수량표현', 'top_prob': '0.988'},
  {'token': '##도를', 'predicted_tag': '기타 수량표현', 'top_prob': '0.9877'},
  {'token': '넘어갈', 'predicted_tag': 'O', 'top_prob': '0.9979'},
  {'token': '것이라고', 'predicted_tag': 'O', 'top_prob': '0.9999'},
  {'token': '예측', 'predicted_tag': 'O', 'top_prob': '0.9999'},
  {'token': '##한다', 'predicted_tag': 'O', 'top_prob': '0.9999'}]}

In [ ]:
sentence = '서울과 울산의 경기 예측을 해줘.'
inference_fn(sentence)

{'sentence': '서울과 울산의 경기 예측을 해줘.',
 'result': [{'token': '서울', 'predicted_tag': '지명', 'top_prob': '0.9146'},
  {'token': '##과', 'predicted_tag': 'O', 'top_prob': '0.9983'},
  {'token': '울산', 'predicted_tag': '지명', 'top_prob': '0.8585'},
  {'token': '##의', 'predicted_tag': 'O', 'top_prob': '0.9972'},
  {'token': '경기', 'predicted_tag': '지명', 'top_prob': '0.5007'},
  {'token': '예측', 'predicted_tag': 'O', 'top_prob': '0.9992'},
  {'token': '##을', 'predicted_tag': 'O', 'top_prob': '0.9998'},
  {'token': '해줘', 'predicted_tag': 'O', 'top_prob': '0.9998'},
  {'token': '.', 'predicted_tag': 'O', 'top_prob': '0.9998'}]}